**Data Exploration & Cleaning**
---
The first part of any data analysis or predictive modeling task is an initial exploration of the data. Even if you collected the data yourself and you already have a list of questions in mind that you want to answer, it is important to explore the data before doing any serious analysis, since oddities in the data can cause bugs and muddle your results. Before exploring deeper questions, you have to answer many simpler ones about the form and quality of data. That said, it is important to go into your initial data exploration with a big picture question in mind since the goal of your analysis should inform how you prepare the data.

In [45]:
# Load in some packages
import calendar
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# **Exploring The Variables**
The first step in exploratory analysis is reading in the data and then exploring the variables. It is important to get a sense of how many variables and cases there are, the data types of the variables and the range of values they take on.

We'll start by reading in the data:

In [46]:
data = pd.read_csv('./datasets/merged/merged.csv') # Read the data

It's a good idea to start off by checking the dimensions of your data set with df.shape and the variable data types of df.dtypes.

In [47]:
data.shape # Check dimensions

(186850, 6)

In [48]:
data.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

The output shows us that we're working with a set of 186850 records and 6 columns. All of the column variables are encoded as "object". Let's check the head of the data to get a better sense of what the variables look like:

In [49]:
data.head(5)  # Check the first 5 rows

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700.0,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


It appears we have a mixture of numeric columns and columns with text data, despite that all of the column variables are encoded as "object". We will convert them if necessary, as we'll see later.

After getting a sense of the data's structure, it is a good idea to look at a statistical summary of the variables with df.describe():

In [50]:
data.describe()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,186305,186305,186305,186305.00,186305,186305
unique,178438,20,18,41.00,142396,140788
top,Order ID,USB-C Charging Cable,1,11.95,Order Date,Purchase Address
freq,355,21903,118804,15358.00,355,355


Describe() give us enough information to determine what each variable means. So, after looking at the data for the first time, you should ask yourself a few questions:

1.   Do I need all of the variables?
2.   Should I transform any variables?
2.   Are there NA values, outliers or other strange values?
2.   Should I create new variables?

# **Do I Need All of the Variables?**

Getting rid of unnecessary variables is a good first step when dealing with any data set, since dropping variables reduces complexity and can make computation on the data faster. Whether you should get rid of a variable or not will depend on size of the data set and the goal of your analysis. With a data set as small as the Titanic data, there's no real need to drop variables from a computing perspective (we have plenty of memory and processing power to deal with such a small data set) but it can still be helpful to drop variables that will only distract from your goal.

From each variable we can take valuable information. In this case, we need all of the variables.

# **Should I Transform Any Variables?**

When you first load a data set, some of the variables may be encoded as data types that don't fit well with what the data really is or what it means.

In [51]:
data['Order Date'].describe()

count         186305
unique        142396
top       Order Date
freq             355
Name: Order Date, dtype: object

Notice that as top frequency for the date feature 'Order Date' is the string value 'Order Date'. This doesn't make any sense, since it should be only date formatted info in this feature. We get rid of it, holding all the rows that don't have 'Order Date' as value in this particular feature:

In [52]:
data = data[data['Order Date'] != 'Order Date']
data.describe()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,185950,185950,185950,185950.00,185950,185950
unique,178437,19,17,40.00,142395,140787
top,160873,USB-C Charging Cable,1,11.95,12/15/19 20:16,"193 Forest St, San Francisco, CA 94016"
freq,5,21903,118804,15358.00,8,9


If you have oddly formatted date time objects, you might have to specify the exact format to get it to convert correctly into a Timestamp. For instance, consider a date format that gives date times of the form hour:minute:second year-day-month:



```
odd_date = "12:30:15 2015-29-11"
```



The default to_datetime parser will fail to convert this date because it expects dates in the form year-month-day. In cases like this, specify the date's format to convert it to Timestamp:

In [53]:
new_Dates = pd.to_datetime(data['Order Date'], format= "%m/%d/%y %H:%M") 
data['Order Date'] = new_Dates

Numerical features 'Quantity Ordered' and 'Price Each' are object types. We should trasform them to numerical type, in order to perform mathematical calculations, with pandas function to_numeric, which converts each feature to the coresponding numerical type:

In [54]:
data['Quantity Ordered'] = pd.to_numeric(data['Quantity Ordered']) # Convert to int
data['Price Each'] = pd.to_numeric(data['Price Each']) # Convert to float

data.describe()

,Quantity Ordered,Price Each
count,185950.000000,185950.000000
mean,1.124383,184.399735
std,0.442793,332.731330
min,1.000000,2.990000
25%,1.000000,11.950000
50%,1.000000,14.950000
75%,1.000000,150.000000
max,9.000000,1700.000000


# **Are there NA Values, Outliers or Other Strange Values?**

Data sets are often littered with missing data, extreme data points called outliers and other strange values. Missing values, outliers and strange values can negatively affect statistical tests and models and may even cause certain functions to fail.

In Python, you can detect missing values with the pd.dropna().

Detecting missing values is the easy part: it is far more difficult to decide how to handle them. In cases where you have a lot of data and only a few missing values, it might make sense to simply delete records with missing values present. On the other hand, if you have more than a handful of missing values, removing records with missing values could cause you to get rid of a lot of data. Missing values in categorical data are not particularly troubling because you can simply treat NA as an additional category. Missing values in numeric variables are more troublesome, since you can't just treat a missing value as number. As it happens, the Sales dataset has some NA's:

In [55]:
data.shape[0] - data.dropna().shape[0]

545

There are 545 NA values, so much smaller number that the original 186495 data. In this case, we will drop all NA values, using pd.dropna().

In [56]:
data.dropna(inplace = True)
data.describe()

,Quantity Ordered,Price Each
count,185950.000000,185950.000000
mean,1.124383,184.399735
std,0.442793,332.731330
min,1.000000,2.990000
25%,1.000000,11.950000
50%,1.000000,14.950000
75%,1.000000,150.000000
max,9.000000,1700.000000


In [57]:
data.describe()

,Quantity Ordered,Price Each
count,185950.000000,185950.000000
mean,1.124383,184.399735
std,0.442793,332.731330
min,1.000000,2.990000
25%,1.000000,11.950000
50%,1.000000,14.950000
75%,1.000000,150.000000
max,9.000000,1700.000000


In [58]:
categorical = data.dtypes[data.dtypes == "object"].index

data[categorical].describe()

,Order ID,Product,Purchase Address
count,185950,185950,185950
unique,178437,19,140787
top,160873,USB-C Charging Cable,"193 Forest St, San Francisco, CA 94016"
freq,5,21903,9


# **Should I Create New Variables?**

The variables present when you load a data set aren't always the most useful variables for analysis. Creating new variables that are derivations or combinations existing ones is a common step to take before jumping into an analysis or modeling task.

For example, imagine you are analyzing web site auctions where one of the data fields is a text description of the item being sold. A raw block of text is difficult to use in any sort of analysis, but you could create new variables from it such as a variable storing the length of the description or variables indicating the presence of certain keywords.

Creating a new variable can be as simple as taking one variable and adding, multiplying or dividing by another. Let's create a new variable, Month, from 'Order Date':

In [59]:
data['Month'] = data['Order Date'].dt.month
data['Month'].describe()

count    185950.000000
mean          7.059140
std           3.502996
min           1.000000
25%           4.000000
50%           7.000000
75%          10.000000
max          12.000000
Name: Month, dtype: float64

Next, we should really create a column featuring 'Sales', which we will create using 'Quantity Ordered' and 'Price Each':

In [60]:
data['Sales'] = data['Quantity Ordered'] * data['Price Each']
data['Sales'].describe()

count    185950.000000
mean        185.490917
std         332.919771
min           2.990000
25%          11.950000
50%          14.950000
75%         150.000000
max        3400.000000
Name: Sales, dtype: float64

From 'Purchase Address' we can take the 'City' that each purchase came from, with apply method:

In [61]:
data['City'] = data['Purchase Address'].apply(lambda x: f"{x.split(',')[1]} ({x.split(',')[2].split(' ')[1]})")
data['City'].describe()

count                  185950
unique                     10
top        San Francisco (CA)
freq                    44732
Name: City, dtype: object

In [62]:
data['Hour'] = data['Order Date'].dt.hour
data['Hour'].describe()

count    185950.000000
mean         14.413305
std           5.423416
min           0.000000
25%          11.000000
50%          15.000000
75%          19.000000
max          23.000000
Name: Hour, dtype: float64

# **Final Dataset**

Finally, we finished exploring and cleaning data. The final dataset is formatted like this:

In [63]:
data

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour
0,141234,iPhone,1.0,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",1,700.00,Boston (MA),21
1,141235,Lightning Charging Cable,1.0,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",1,14.95,Portland (OR),14
2,141236,Wired Headphones,2.0,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",1,23.98,San Francisco (CA),13
3,141237,27in FHD Monitor,1.0,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",1,149.99,Los Angeles (CA),20
4,141238,Wired Headphones,1.0,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",1,11.99,Austin (TX),11
...,...,...,...,...,...,...,...,...,...,...
185945,319666,Lightning Charging Cable,1.0,14.95,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",12,14.95,San Francisco (CA),20
185946,319667,AA Batteries (4-pack),2.0,3.84,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",12,7.68,Los Angeles (CA),12
185947,319668,Vareebadd Phone,1.0,400.00,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",12,400.00,Seattle (WA),6
185948,319669,Wired Headphones,1.0,11.99,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",12,11.99,Dallas (TX),10


And the last step is to save this dataset as csv file.

In [64]:
data.to_csv('sales.csv', index=False)